# CUDA编程模型---矩阵乘法

#### 通过向量加法，我们已经学会了如何调用线程。接下来，我们来实践一下，如何利用Cuda处理矩阵。今天的课程将会介绍：
1. 二维矩阵的乘法
2. 二维矩阵的转置
3. 如何分配线程和GPU存储单元

### 1.矩阵乘法是科学计算和深度学习领域常见的操作，我们先来看一看CPU代码如何处理矩阵乘法
```c
void cpu_matrix_mult(int *h_a, int *h_b, int *h_result, int m, int n, int k) 
{
    for (int i = 0; i < m; ++i) 
    {
        for (int j = 0; j < k; ++j) 
        {
            int tmp = 0;
            for (int h = 0; h < n; ++h) 
            {
                tmp += h_a[i * n + h] * h_b[h * k + j];
            }
            h_result[i * k + j] = tmp;
        }
    }
}
```
这时，我们看到在CPU代码中，需要嵌套三个for循环，也就是说CPU的线程会一个接一个的求结果矩阵中的每一个数值，直到处理完所有数值。那么，我们在GPU中就可以申请很多个线程，每个线程来求结果矩阵中的一个数值，并同时完成

![matrix_mul](matrix_mul.png)

那么，首先我们要得到每一个执行线程，在Grid所有线程中的(x,y)坐标，如下图所示，即(Thread_x, Thread_y)

![](matrix_mul2.png)

也就是说，以上面的CPU代码为例，我们要让编号为(Thread_x, Thread_y)的线程读取a矩阵中的一行和b矩阵中的一列，然后把对应元素乘积并累加。

接下来我们要考虑另一个问题: 如何将二维矩阵的坐标映射到一维.

我们知道二维矩阵实际在计算机系统中, 也是以一维连续地址进行存储的, 如下图所示: 

![](array_2to1.png)

那么, 我们假设一个矩阵的宽高分别为: `width` 和`height`, 那么一个坐标为(x,y)的元素, 他在该矩阵的一维索引值就应该是:

`y * width + x`

**注意: 通常我们定义一个矩阵为m * n, 那么表示它有m行, n列. 对应到上面是, width == n && height == m**

明白了上述原理, 我们就可以很轻松的通过CUDA线程中的索引值, 找到需要处理的数据

接下来，我们在[matrix_mul.cu](matrix_mul.cu)中完成这一过程，如果遇到麻烦，请参考[result_1.cu](result_1.cu)

In [ ]:
!/usr/local/cuda/bin/nvcc matrix_mul.cu -o matrix_mul

执行，并查看结果是否正确

In [ ]:
!./matrix_mul

利用nvprof来查看程序性能

In [ ]:
!sudo /usr/local/cuda/bin/nvprof ./matrix_mul

修改矩阵大小为1000*1000，并查看效果

### 2.矩阵转置

矩阵转置也是在众多科学计算中常用的方法, 我们先来看下CPU如何处理二维矩阵转置:

```c
void cpu_matrix_transpose(int in[N][M], int out[M][N])
{
    for(int y = 0; y < N; y++)
    {
        for(int x = 0; x < M; x++)
        {
            out[x][y] = in[y][x];
        }
    }
}
```

接下来, 我们尝试用GPU加速这一过程

同学们在[transpose.cu](transpose.cu)中完成这一过程，如果遇到麻烦，请参考[result_2.cu](result_2.cu)


In [ ]:
!/usr/local/cuda/bin/nvcc transpose.cu -o transpose

In [ ]:
!./transpose

课后作业：
1. 当我们能申请的线程数很少，远远不够的时候怎么办？
2. 修改[im2gray.cu](im2gray.cu), 完成将RGB图像转化为灰度图的程序. 如果遇到麻烦, 请参考[im2gray_result.cu](im2gray_result.cu)